In [1]:
import pandas as pd
from google.cloud import bigquery

# Set your project and dataset IDs.
PROJECT_ID = "rocketech-de-pgcp-sandbox"
DATASET_ID = "ecommerce_analytics"
TABLE_PREFIX = f"{PROJECT_ID}.{DATASET_ID}"

client = bigquery.Client(project=PROJECT_ID)

**Query 1: Daily Order Count and Total Revenue Last 7 days**

In [ ]:
print("--- Executing: Query 1: Daily Order Count and Total Revenue Last 7 days ---")

query_1 = f"""
SELECT
  DATE(order_date_2) AS order_date_correct,
  COUNT(DISTINCT order_id) AS total_orders,
  SUM(order_total_usd) AS total_revenue
FROM
  `{TABLE_PREFIX}.orders`
WHERE
  order_date_2 >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 7 DAY)
GROUP BY
  DATE(order_date_2)
ORDER BY
  DATE(order_date_2) DESC;
"""

try:
    df1 = client.query(query_1).to_dataframe()
except Exception as e:
    print(f"Error converting query results to DataFrame: {e}")

df1

**Query 2: Top 10 Customers by Total Spending**

In [ ]:
print("--- Executing: Query 2: Top 10 Customers by Total Spending ---")

query_2 = f"""
SELECT
  t1.customer_id,
  t1.first_name,
  t1.last_name,
  SUM(t2.order_total_usd) AS lifetime_spending
FROM
  `{TABLE_PREFIX}.customers` AS t1
JOIN
  `{TABLE_PREFIX}.orders` AS t2
ON
  t1.customer_id = t2.cust_acct_id
GROUP BY
  t1.customer_id, t1.first_name, t1.last_name
ORDER BY
  lifetime_spending DESC
LIMIT 10;
"""

try:
    df2 = client.query(query_2).to_dataframe()
except Exception as e:
    print(f"Error converting query results to DataFrame: {e}")

df2

**Query 3: Total Revenue by Product Category from the past 7 days**

In [ ]:
print("--- Executing: Query 3: Total Revenue by Product Category from the past 7 days ---")

query_3 = f"""
SELECT
  t1.category_name,
  SUM(t3.line_item_total) AS total_category_revenue
FROM
  `{TABLE_PREFIX}.product_category` AS t1
JOIN
  `{TABLE_PREFIX}.products` AS t2
ON
  t1.category_id = t2.prod_cat_id
JOIN
  `{TABLE_PREFIX}.order_item` AS t3
ON
  t2.product_id = t3.item_id
JOIN
  `{TABLE_PREFIX}.orders` AS t4
ON
  t3.order_num = t4.order_id
WHERE
  t4.order_date_2 >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 7 DAY)
GROUP BY
  t1.category_name
ORDER BY
  total_category_revenue DESC;
"""

try:
    df3 = client.query(query_3).to_dataframe()
except Exception as e:
    print(f"Error converting query results to DataFrame: {e}")

df3

**Query 4: Top 5 Most Sold Products**

In [ ]:
print("--- Executing: Query 4: Top 5 Most Sold Products ---")

query_4 = f"""
SELECT
  t1.product_name,
  t1.prod_cat_id,
  SUM(t2.quantity) AS total_quantity_sold
FROM
  `{TABLE_PREFIX}.products` AS t1
JOIN
  `{TABLE_PREFIX}.order_item` AS t2
ON
  t1.product_id = t2.item_id
GROUP BY
  t1.product_name, t1.prod_cat_id
ORDER BY
  total_quantity_sold DESC
LIMIT 5;
"""

try:
    df4 = client.query(query_4).to_dataframe()
except Exception as e:
    print(f"Error converting query results to DataFrame: {e}")

df4

**Average Shipping Time by Signup Month**

In [ ]:
print("--- Executing: Query 5: Average Shipping Time by Signup Month ---")

query_5 = f"""
SELECT
  FORMAT_DATE('%Y-%m', t1.signup_date) AS customer_signup_month,
  AVG(DATE_DIFF(t2.delivery_date, t2.shipping_date, DAY)) AS avg_shipping_time_days
FROM
  `{TABLE_PREFIX}.customers` AS t1
JOIN
  `{TABLE_PREFIX}.orders` AS t2
ON
  t1.customer_id = t2.cust_acct_id
WHERE
  t2.shipping_date IS NOT NULL
  AND t2.delivery_date IS NOT NULL
GROUP BY
  customer_signup_month
ORDER BY
  customer_signup_month;
"""

try:
    df5 = client.query(query_5).to_dataframe()
except Exception as e:
    print(f"Error converting query results to DataFrame: {e}")

df5